# Forecasting Malaysia's Total Export Value with Machine Learning Regression Models

## Import libraries and packages

In [11]:
import requests
import pandas as pd
import json
import os
import re
import time
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import logging
from airflow.exceptions import AirflowFailException
from bs4 import BeautifulSoup
from bs4.element import Tag
from fredapi import Fred
from imfpy import searches, retrievals
from typing import Optional
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
from dotenv import load_dotenv
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima import auto_arima
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn import metrics #to evaluate the accuracy of our ml model
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from pmdarima.arima import auto_arima
from lazypredict.Supervised import LazyRegressor

## ETL Process
### Malaysia's Total Export and Import Value from METS OnlineFred

#### Functions for Google Cloud

In [2]:
# Define a function for uploading data to Google Storage Bucket    
def upload_to_bucket(storage_client,
                     bucket_name,
                     df_list):
    if df_list is None:
        logging.warning("Please ensure df_list has a value.")
    
    my_bucket = storage_client.bucket(bucket_name)
    # Check if the specified bucet_name exists or not
    if not my_bucket.exists(): # If the bucket does not exist
        try:
            logging.info(f'Bucket - {bucket_name} is not found.')
            logging.info(f"Creating {bucket_name} in progress ...")
            my_bucket.create() # Create the bucket with the specified name
            logging.info(f"SUCCESS: {bucket_name} has been created.")
            
        except Exception as e:
            logging.info(f"Error creating bucket: {e}")
                  
    else:
        logging.info(f"Bucket - {bucket_name} is found.")
   
    try:
        # To create a list to store the gsutil uri 
        gsutil_uri_list = []
        for item in df_list:
            # timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S ')
            # blob_name = item.name + " " + timestamp
            blob_name = item.name
            blob = my_bucket.blob(blob_name)
            logging.info(f"Uploading data to Google Storage Bucket in progress ...")
            blob.upload_from_string(item.to_csv(index=False), 'text/csv')
            logging.info(f'SUCCESS: {blob} has successfully uploaded to {my_bucket}.')
            gsutil_uri = f"gs://{bucket_name}/{blob_name}"
            gsutil_uri_list.append(gsutil_uri) 
        return gsutil_uri_list

    except Exception as e:
        logging.error(f"Error: {e}")

# Define a function to load data from google bucket to google bigquery
def upload_to_bigquery(client, dataset_name, table_name, job_config, gsutil_uri):
    try:
        for i in range(len(table_name)): 
            # Look up for dataset
            dataset_ref = client.dataset(dataset_name)
            try:
                targeted_dataset = client.get_dataset(dataset_ref)
                logging.info(f'Dataset {targeted_dataset.dataset_id} already exists.')
            # Create new dataset if not found
            except NotFound:
                logging.info(f"Dataset {dataset_ref} is not found")
                logging.info(f"Creating dataset - {dataset_ref} in progress ...")
                targeted_dataset = client.create_dataset(dataset_ref)
                logging.info(f'Dataset {targeted_dataset.dataset_id} created.')

            # Look up for table
            table_ref = dataset_ref.table(table_name[i])
            try:
                targeted_table = client.get_table(table_ref)
                logging.info(f'Table {targeted_table.table_id} already exists.')
            # Create new table if not found
            except NotFound:
                logging.info(f"Dataset {table_ref} is not found")
                logging.info(f"Creating table - {table_ref} in progress ...")
                targeted_table = client.create_table(table_ref)
                logging.info(f'Table {targeted_table.table_id} created.')

            # Upload the data to bigquery table using gsutil URI
            load_job = client.load_table_from_uri(gsutil_uri[i],
                                                  targeted_table, 
                                                  job_config=job_config)

            logging.info(load_job.result())
            logging.info(f"SUCCESS: The data has been loaded to Google BigQuery.")
        
    except Exception as e:
        logging.error(f"Error: {e}")
        
def create_blob_list(storage_client, 
                     bucket_name, 
                     df_list):
    if df_list is None:
        logging.warning("Please ensure df_list has a value.")
        exit()
    
    my_bucket = storage_client.bucket(bucket_name)
    # Check if the specified bucet_name exists or not
    if not my_bucket.exists(): # If the bucket does not exist
        try:
            logging.info(f'Bucket - {bucket_name} is not found.')
            logging.info(f"Creating {bucket_name} in progress ...")
            my_bucket.create() # Create the bucket with the specified name
            logging.info(f"SUCCESS: {bucket_name} has been created.")
            
        except Exception as e:
            logging.error(f"Error creating bucket: {e}")
                  
    else:
        logging.info(f"Bucket - {bucket_name} is found.")
    
    try:
        # To create a list to store all the blobs uploaded to the bucket
        blob_list =[]
        for item in df_list:
            blob_name = item.name
            blob = my_bucket.blob(blob_name)
            logging.info(f"Uploading data to Google Storage Bucket in progress ...")
            blob.upload_from_string(item.to_csv(index=False), 'text/csv')
            logging.info(f"SUCCESS: {blob} has been uploaded to {my_bucket}.")
            blob_list.append(blob_name)
        return blob_list
            
    except Exception as e:
        logging.error(f"Error: {e}")

#### Functions for METS ETL

In [3]:
# Define a function for extracting raw data from METS Online
def mets_extract(url,
                 payload: Optional[dict] = None,
                 headers: Optional[dict] = None):
    
    # Specify the maximum number of retry
    MAX_RETRIES = 5
    
    # Define the retry strategy
    retry_strategy = Retry(total = MAX_RETRIES,
                           backoff_factor = 2,
                           status_forcelist = [429, 500, 502, 503, 504],
                           allowed_methods = ["GET", "POST"])
    
    # Create an HTTP adapter with the retry strategy and mount it to session
    adapter = HTTPAdapter(max_retries=retry_strategy)
    
    # Create a new session object
    session = requests.Session()
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    logging.info("Extracting raw data in progress ...")
    # Make a request using the created session object
    raw_data = session.post(url,
                            data = payload,
                            headers = headers,
                            verify = False)
    
    if raw_data.status_code == 200:
        logging.info('SUCCESS: Raw Data has been extracted.')

        # Parse the HTML
        raw_data = BeautifulSoup(raw_data.text, 'html.parser')
        return raw_data

    else:
        logging.error(f'Error: Fail to extract the raw data. ErrorCode: {raw_data.status_code}.')

# Define a function for basic preprocessing on the extracted raw html text.
def mets_preprocess(raw_data, 
                    dataframe_name):
    
    # Look up for the table
    result = raw_data.find('table', class_='table-bordered')
    
    # Extract table rows
    rows = result.find_all('tr')
    
    individual_data = []
    for row in rows:
        data = row.find_all(['th', 'td'])
        if data:
            data = [item.get_text(strip=True) for item in data]
            if 'GRAND TOTAL' not in data:
                individual_data.append(data)
    
    logging.info('Converting raw data to dataframe......')
    # Select a subset of columns from the first row as column names
    df = pd.DataFrame(individual_data[1:], columns=individual_data[0])
    # To remove the yearly data column due to redundancy 
    data_month_filter = [col for col in df.columns if '-' not in col or not any(char.isdigit() for char in col)]
    df_list = []
    df_monthly = df.loc[:, data_month_filter]
    df_monthly.name = dataframe_name
    df_list.append(df_monthly)
    logging.info(f'SUCCESS: {dataframe_name} has been created')
    
    return df_list

def mets_transformation(df_list, 
                        new_column_name, 
                        dataframe_name):
    
    transformed_df_list = []
    replacements = {' ': '_',
                    '&': '',
                    ',': '',
                    '.': '',}
    try:
        for item in df_list:
            df = item.transpose().reset_index()
            column_name = list(df.iloc[2,:])
            edited_column_name = []

            # To replace the selected symbols and empty spaces
            for column in column_name:
                for old, new in replacements.items():
                    column = column.replace(old, new)
                edited_column_name.append(column)

            df = df.iloc[3:]
            df.columns = edited_column_name
            df = df.rename(columns={'PRODUCT_DESCRIPTION': 'date'})

            # To convert the 'date' column to datetime format, whereas the others are converted to numeric.
            for column in df.columns:
                if column == 'date':
                    df[column] = pd.to_datetime(df[column])
                else:
                    df[column] = pd.to_numeric(df[column].str.replace(',', ''))

            # Create a new column which sum all the values from other columns in the same row
            df[new_column_name] = df.iloc[:,1:].sum(axis=1)
            df.name = dataframe_name

            transformed_df_list.append(df)
            logging.info(f"SUCCESS: Dataframe - {dataframe_name} has been transformed.")
            return transformed_df_list
    except Exception as e:
        logging.error(f"Error: {e}")

#### Input parameters

In [4]:
load_dotenv()

# Retrieve the csrf_token and Cookie values
csrf_token = os.getenv("csrf_token")
Cookie = os.getenv("cookie")

# URL of targeted site
url = "https://metsonline.dosm.gov.my/tradev2/product-coderesult"

# headers of targeted site
headers = {
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Cookie": "PHPSESSID=938e8fjm1hj9bb07oq3u4fisi4; _csrf=615b79fbeb5a5e5147fee3ac2245a8abb4f3d106b51ba953154dfe4b8036f358a%3A2%3A%7Bi%3A0%3Bs%3A5%3A%22_csrf%22%3Bi%3A1%3Bs%3A32%3A%22V4KGkyVFWmBgtSAZPHpnC8MV4N1-eYuB%22%3B%7D; cookiesession1=678B288423AE9070F6271BFC1DD8DA77"
}

# payload of targeted site
payload_exports = {
    "_csrf": "Q3RsVE1CS1QVQCcTJjsdEhQZLjM5EQoOEzwcOg56BgJ3Ol15KBs.Fg==",
    "Tradev2[typeofsearch]": "classification",
    "Tradev2[typedigit]": 7,
    "Tradev2[rangecode1]": 0,
    "Tradev2[rangecode2]": 9,
    # 'Tradev2[code_idcode]': ,
    # 'Tradev2[code_idcodedigit9]': ,
    # 'Tradev2[tradeflow]': ,
    "Tradev2[tradeflow][]": "exports",
    # 'Tradev2[timeframe]': ,
    "Tradev2[timeframe]": "month",
    # 'Tradev2[rangeyear]': ,
    # 'Tradev2[rangeyear2]': ,
    # 'Tradev2[rangeyearone]': ,
    # 'Tradev2[rangemonthone]': ,
    "Tradev2[mothdata]": 2000,
    "Tradev2[mothdata2]": 2001,
    # 'Tradev2[classification_serch]': ,
    # 'Tradev2[country2]': ,
    "Tradev2[geogroup]": 1,
    "Tradev2[geogroup]": 29,
    "Tradev2[codeshowby]": "code",
}

# payload of targeted site
payload_imports = {
    "_csrf": "Q3RsVE1CS1QVQCcTJjsdEhQZLjM5EQoOEzwcOg56BgJ3Ol15KBs.Fg==",
    "Tradev2[typeofsearch]": "classification",
    "Tradev2[typedigit]": 7,
    "Tradev2[rangecode1]": 0,
    "Tradev2[rangecode2]": 9,
    # 'Tradev2[code_idcode]': ,
    # 'Tradev2[code_idcodedigit9]': ,
    # 'Tradev2[tradeflow]': ,
    "Tradev2[tradeflow][]": "imports",
    # 'Tradev2[timeframe]': ,
    "Tradev2[timeframe]": "month",
    # 'Tradev2[rangeyear]': ,
    # 'Tradev2[rangeyear2]': ,
    # 'Tradev2[rangeyearone]': ,
    # 'Tradev2[rangemonthone]': ,
    "Tradev2[mothdata]": 2000,
    "Tradev2[mothdata2]": 2001,
    # 'Tradev2[classification_serch]': ,
    # 'Tradev2[country2]': ,
    "Tradev2[geogroup]": 1,
    "Tradev2[geogroup]": 29,
    "Tradev2[codeshowby]": "code",
}
# Google Cloud Parts
# Convert the credentials to .json file for the usage of GOOGLE_APPLICATION_CREDENTIALS
CREDENTIALS = json.loads(os.getenv('CREDENTIALS'))

# Check if there's an existing credentials file
with open('credentials.json', 'w') as cred_file:
    json.dump(CREDENTIALS, cred_file)
    
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='credentials.json'

storage_client = storage.Client()
bucket_name = 'trial-malaysia-international-trade'
bq_client = bigquery.Client()
dataset_name = "trial_area"
table_name_export = ["trial_export_my"]
table_name_import = ["trial_import_my"]
job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.CSV,
                                    write_disposition='WRITE_TRUNCATE',
                                    skip_leading_rows=1,
                                    autodetect=True,)

#### Test Run for Exports

In [5]:
raw_data_exports = mets_extract(url=url, payload=payload_exports, headers=headers)
df_list = mets_preprocess(raw_data=raw_data_exports, dataframe_name='export_trial')
transformed_df_list = mets_transformation(df_list=df_list, new_column_name="total_export_value", dataframe_name="total_export")
gsutil_uri_list = upload_to_bucket(storage_client=storage_client,
                                   bucket_name=bucket_name,
                                   df_list=transformed_df_list)
upload_to_bigquery(client=bq_client,
                   dataset_name=dataset_name,
                   table_name=table_name_export,
                   job_config=job_config,
                   gsutil_uri=gsutil_uri_list)

[2024-02-06T09:41:00.688+0800] {1547133257.py:23} INFO - Extracting raw data in progress ...
[2024-02-06T09:41:25.664+0800] {1547133257.py:31} INFO - SUCCESS: Raw Data has been extracted.
[2024-02-06T09:41:25.711+0800] {1547133257.py:58} INFO - Converting raw data to dataframe......
[2024-02-06T09:41:25.714+0800] {1547133257.py:67} INFO - SUCCESS: export_trial has been created
[2024-02-06T09:41:25.724+0800] {1547133257.py:108} INFO - SUCCESS: Dataframe - total_export has been transformed.
[2024-02-06T09:44:14.445+0800] {1887242903.py:21} INFO - Bucket - trial-malaysia-international-trade is found.
[2024-02-06T09:44:14.446+0800] {1887242903.py:31} INFO - Uploading data to Google Storage Bucket in progress ...
[2024-02-06T09:44:14.881+0800] {1887242903.py:33} INFO - SUCCESS: <Blob: trial-malaysia-international-trade, total_export, 1707183853058936> has successfully uploaded to <Bucket: trial-malaysia-international-trade>.
[2024-02-06T09:46:00.778+0800] {1887242903.py:49} INFO - Dataset t

#### Test run for Imports

In [6]:
raw_data_imports = mets_extract(url=url, payload=payload_imports, headers=headers)
df_list = mets_preprocess(raw_data=raw_data_imports, dataframe_name='import_trial')
transformed_df_list = mets_transformation(df_list=df_list, new_column_name="total_import_value", dataframe_name="total_import")
gsutil_uri_list = upload_to_bucket(storage_client=storage_client,
                                   bucket_name=bucket_name,
                                   df_list=transformed_df_list)
upload_to_bigquery(client=bq_client,
                   dataset_name=dataset_name,
                   table_name=table_name_import,
                   job_config=job_config,
                   gsutil_uri=gsutil_uri_list)

[2024-02-06T09:46:04.240+0800] {1547133257.py:23} INFO - Extracting raw data in progress ...
[2024-02-06T09:46:29.556+0800] {1547133257.py:31} INFO - SUCCESS: Raw Data has been extracted.
[2024-02-06T09:46:29.592+0800] {1547133257.py:58} INFO - Converting raw data to dataframe......
[2024-02-06T09:46:29.594+0800] {1547133257.py:67} INFO - SUCCESS: import_trial has been created
[2024-02-06T09:46:29.605+0800] {1547133257.py:108} INFO - SUCCESS: Dataframe - total_import has been transformed.
[2024-02-06T09:46:29.810+0800] {1887242903.py:21} INFO - Bucket - trial-malaysia-international-trade is found.
[2024-02-06T09:46:29.812+0800] {1887242903.py:31} INFO - Uploading data to Google Storage Bucket in progress ...
[2024-02-06T09:46:30.246+0800] {1887242903.py:33} INFO - SUCCESS: <Blob: trial-malaysia-international-trade, total_import, 1707183988428578> has successfully uploaded to <Bucket: trial-malaysia-international-trade>.
[2024-02-06T09:46:30.678+0800] {1887242903.py:49} INFO - Dataset t

### USDMYR Exchange Rate and RBEER from Federal Reserve Economic Data (FRED)

#### Functions for FRED ETL

In [7]:
def fred_data_extraction(selected_data):
    df_list = []
    for item in selected_data:
        logging.info(f"Extracting {item} data in progress ...")
        df = fred.get_series_all_releases(item)
        logging.info(f"SUCCESS: {item} data has been extracted.")
        df.name = item
        df_list.append(df)
    return df_list

#### Test Run for FRED

In [8]:
FRED_API = os.getenv("FRED_API")
fred = Fred(api_key = FRED_API)

In [9]:
selected_data = ['EXMAUS', 'RBMYBIS']
df_list = fred_data_extraction(selected_data=selected_data)
gsutil_uri_list = upload_to_bucket(storage_client=storage_client,
                                   bucket_name=bucket_name,
                                   df_list=df_list)
upload_to_bigquery(client=bq_client,
                   dataset_name=dataset_name,
                   table_name=selected_data,
                   job_config=job_config,
                   gsutil_uri=gsutil_uri_list)

[2024-02-06T09:46:33.877+0800] {1846224335.py:4} INFO - Extracting EXMAUS data in progress ...
[2024-02-06T09:46:34.571+0800] {1846224335.py:6} INFO - SUCCESS: EXMAUS data has been extracted.
[2024-02-06T09:46:34.572+0800] {1846224335.py:4} INFO - Extracting RBMYBIS data in progress ...
[2024-02-06T09:46:36.041+0800] {1846224335.py:6} INFO - SUCCESS: RBMYBIS data has been extracted.
[2024-02-06T09:46:36.242+0800] {1887242903.py:21} INFO - Bucket - trial-malaysia-international-trade is found.
[2024-02-06T09:46:36.243+0800] {1887242903.py:31} INFO - Uploading data to Google Storage Bucket in progress ...
[2024-02-06T09:46:36.678+0800] {1887242903.py:33} INFO - SUCCESS: <Blob: trial-malaysia-international-trade, EXMAUS, 1707183994863265> has successfully uploaded to <Bucket: trial-malaysia-international-trade>.
[2024-02-06T09:46:36.679+0800] {1887242903.py:31} INFO - Uploading data to Google Storage Bucket in progress ...
[2024-02-06T09:46:37.540+0800] {1887242903.py:33} INFO - SUCCESS: <

### World Total Export Value from Inernational Monetary Fund (IMF)

#### Functions for IMF ETL

In [19]:
def dots(country, counterparts, start, end, freq='A', form="wide"):
    
    """
    Highly flexible function to return time series trade data between countries from the IMF Direction of Trade (DOTS) Database.
    The function sends a get request to the IMF JSON RESTful API. 

    Parameters
    ----------
    country : str (required)
        Country code for home country. 
        Use searches.country_codes() for a list of codes.
        Use searches.country_search("keyword") to search countries
    counterparts : str or list (required)
        Country code(s) for the counterpart country (or countries)
        Use searches.country_codes() for a list of codes.
        Use searches.country_search("keyword") to search countries
    start: int or float (reqiured)
        Start date of the series. 
        Years may be entered as int dtype, such as 1980
        Month start dates, such as 1980.02 (float) are acceptable.
        Numbers such as 1980.00 and 1980.14 are unacceptable.
    end: int or float (required)
        Same as start date but end of the series.
    freq: str (optional, default='A')
        Frequency of the time series (intervals)
        Default: 'A' - annual
        Alternatives: 'M' - monthly
        Note, freq "A" will override start dates entered as months, such as 1980.02
        In this case, start will be rounded down to the nearest whole year
        And end will be rounded up to the nearest whole year
    form: str (optional, default='A')
        If multiple counterparts, should the returned data be wide-form or long-form?
        Default: 'wide' (MultiIndex)
        Alternatives: 'long'

    Returns
    -------
    full_df : pandas.core.frame.DataFrame
        DataFrame with trade statistics.
        If multiple counterpart countries are selected and wide-form data is requested,
        the resulting DataFrame will be multiIndexed/hierarchical

    Examples
    --------
    >>> dots('US', 'CN', 1995, 2020)
    Returns wide-form US-China annual data between 1995 and 2020.
    
    >>> dots('MX','W00', 2010, 2020, freq='M')
    Returns wide-form Mexico-World monthly data between 2010 and 2020.
    
    >>> dots("GR", ["US", "AU", "DE"], 1998, 2018)
    Returns wide-form Greece annual data vs. the U.S., Australia and Germany 
    Between 1998 and 2018.
        
    >>> dots("XS25", ["JP", "KR"], 2000.05, 2020.09, freq="M", form="long")
    Returns long-form monthly data from Developing Asia vs. Japan and Korea
    Between May 2005 and September 2009

    """
    #validate input datatypes
    assert isinstance(country, str), "country must be a str"
    assert isinstance(counterparts, (str, list)), "counterparts must be a str or list"
    # if isinstance(counterparts, list):
    #     assert len(counterparts) > 1, "counterparts must be a str or list of length 2 or more"
    #     assert country not in counterparts, "country must not be in counterparts"
    # else:
    #     assert country != counterparts, "country and counterpart must not be the same"
    assert isinstance(start, (int,float)),"start must be a number"
    assert isinstance(end, (int,float)), "end must be a number"
    assert freq=="M" or freq=="A", "frequency must be M or A"
    assert form in ['long', 'wide'], "form must be long or wide"
    assert start > 1800 and start < 2200, "start must be a reasonable date"
    assert end > 1800 and end < 2200, "end must be a reasonable date"
    assert end >= start, "end must be after start"
    
    #transform mismatchedfrequency and start/end dates, if applicable
    if freq=="A" and isinstance(start, float):
        start = int(start)
    if freq=="A" and isinstance(end, float):
        end = int(end)+1
    
    #import libraries and define base URL for API
    import requests, dateutil.parser
    import pandas as pd
    start_url = "http://dataservices.imf.org/REST/SDMX_JSON.svc/"
    
    #Specify all available series for trade (exports, imports and trade balance)
    series = 'TBG_USD+TXG_FOB_USD+TMG_CIF_USD' 
    
    #define subfunction to handle single requests
    def retrieve(counterpart):
        
        request = f'{start_url}CompactData/DOT/{freq}.{country}.{series}.{counterpart}?startPeriod={start}&endPeriod={end}'

        #Send the get request to the API
        r = requests.get(request)
        print(r)
        
        #assert the response was 200 (OK)
        assert r.status_code==200, "Error - HTTP Request unsuccessful. Please try again."
            
        #convert the data to subscriptable json 
        data_json = r.json()
        
        try:
            #extract exports, imports and trade balance portions of the JSON (they are lists)
            exports = data_json['CompactData']['DataSet']['Series'][0]['Obs']
            imports = data_json['CompactData']['DataSet']['Series'][1]['Obs']
            tbal = data_json['CompactData']['DataSet']['Series'][2]['Obs']
            
        #if series is not found, throw an error.    
        except KeyError:
            raise AssertionError("One or more series not found. Please try again.")

        #Make sure all series are the same length
        assert len(exports)==len(imports), "Error - data not available. Try a different time period or frequency."
    
        #Parse time periods and values for each series
        periods = [dateutil.parser.parse(obs['@TIME_PERIOD']) for obs in exports]
        values_exports = [float(obs['@OBS_VALUE']) for obs in exports]
        values_imports = [float(obs['@OBS_VALUE']) for obs in imports]
        values_tbal = [float(obs['@OBS_VALUE']) for obs in tbal]
  
        #Convert to a pandas dataframe
        compile_df = pd.DataFrame({'Period':periods,
                                'Exports':values_exports, 
                                'Imports':values_imports, 
                                'Trade Balance':values_tbal})
        
        #Inlucde a column for two-way trade (exports + imports)
        compile_df['Twoway Trade']=compile_df['Exports']+compile_df['Imports']
    
        #Return the dataframe
        return compile_df
    
    #subfunction to format date correctly based on user input frequency
    def format_date(full_df):
        if freq=="A":
            full_df['Period'] = full_df['Period'].apply('{:%Y}'.format)
        else:
            full_df['Period'] = full_df['Period'].apply('{:%Y-%m}'.format)  
        return full_df
    
    #if counterparts is a list of countries, send a request for each country
    #append it to a master dataframe in long form
    #pivot to wide and return
    if isinstance(counterparts, list):
        
        full_df = pd.DataFrame()
        for counterpart in counterparts:
            retrieved = retrieve(counterpart)
            retrieved.insert(1,"Counterpart",counterpart)
            full_df = full_df.append(retrieved) #if long-form data requested, stop here
        
        #format dates correctly based on the user-specified frequency
        full_df = format_date(full_df)
        
        if(form=="wide"): #otherwise, pivot to wide form data
            full_df.insert(1,"Country",country)
            full_df = full_df.pivot(index="Period",
                    columns='Counterpart', 
                    values=['Exports', 'Imports', 'Trade Balance', 'Twoway Trade'])
            full_df.insert(0,"Country",country)
        else:
            full_df.insert(1,"Country",country) #if long-form data, insert country at position 1
        
    #if counterparts is a single country, create columns for country, counterpart
    #and return the result of that single request
    else:
        full_df = retrieve(counterparts)
        full_df.insert(1,'Country',country)
        full_df.insert(2,'Counterpart',counterparts)
        
        #format dates correctly based on the user-specified frequency
        full_df = format_date(full_df)
        
    return full_df

def imf_extraction(countries, 
                   counterparts, 
                   start_year, 
                   end_year, 
                   frequency,):
    
    df_list = []
    try:
        for i in range(min(len(countries), len(counterparts))):
            country = countries[i]
            counterpart = counterparts[i]
            logging.info(f"Extracting {country}-{counterpart} data in progress ...")
            df = dots(country, counterpart, start_year, end_year, frequency)
            logging.info(f"SUCCESS: {country}-{counterpart} data has been extracted.")
            df.name = countries[i] + "-" + counterparts[i]
            df_list.append(df)
            return df_list
    except Exception as e:
            logging.error(f"Error: {e}")

#### Test Run for IMF

In [17]:
countries = ['W00']
counterparts = ['W00']

In [20]:
df_list = imf_extraction(countries=countries,
                         counterparts=counterparts,
                         start_year=2000,
                         end_year=2023,
                         frequency="M",)
gsutil_uri_list = upload_to_bucket(storage_client=storage_client,
                                   bucket_name=bucket_name,
                                   df_list=df_list)
upload_to_bigquery(client=bq_client,
                   dataset_name=dataset_name,
                   table_name=['world_export'],
                   job_config=job_config,
                   gsutil_uri=gsutil_uri_list)

[2024-02-06T10:06:22.773+0800] {735304471.py:190} INFO - Extracting W00-W00 data in progress ...
<Response [200]>
[2024-02-06T10:06:23.371+0800] {735304471.py:192} INFO - SUCCESS: W00-W00 data has been extracted.
[2024-02-06T10:06:23.568+0800] {1887242903.py:21} INFO - Bucket - trial-malaysia-international-trade is found.
[2024-02-06T10:06:23.568+0800] {1887242903.py:31} INFO - Uploading data to Google Storage Bucket in progress ...
[2024-02-06T10:06:23.927+0800] {1887242903.py:33} INFO - SUCCESS: <Blob: trial-malaysia-international-trade, W00-W00, 1707185182199680> has successfully uploaded to <Bucket: trial-malaysia-international-trade>.
[2024-02-06T10:06:24.369+0800] {1887242903.py:49} INFO - Dataset trial_area already exists.
[2024-02-06T10:06:24.751+0800] {1887242903.py:61} INFO - Table world_export already exists.
[2024-02-06T10:06:27.698+0800] {1887242903.py:74} INFO - LoadJob<project=personal-project-401309, location=US, id=414749cc-2244-466b-8c79-5de63f68ba2f>
[2024-02-06T10:0

In [10]:
# To merge df1 and df2
final_df = pd.merge(df1, df2)

# New column for MY export in USD currency and remove the initial export value
final_df['Export'] = final_df['export']/final_df['ER']
final_df = final_df.drop('export', axis = 1)

# To assign units to the dataset
final_df['Pop'] = final_df['Pop']/10**6 # in millions
final_df['GDP'] = final_df['GDP']/10**9 # in billions
final_df['Export'] = final_df['Export']/10**9 # in billions

display(final_df.head())
final_df.info()

NameError: name 'df1' is not defined

## Check Stationarity of Data
## Unit Root Test - Augmented Dickey-Fuller (ADF) test

In [ ]:
f, ax = plt.subplots(nrows=3, ncols=2, figsize=(15, 9))

def visualize_adfuller_results(series, title, ax):
    result = adfuller(series)
    significance_level = 0.05
    adf_stat = result[0]
    p_val = result[1]
    crit_val_1 = result[4]['1%']
    crit_val_5 = result[4]['5%']
    crit_val_10 = result[4]['10%']

    if (p_val < significance_level) & ((adf_stat < crit_val_1)):
        linecolor = 'forestgreen' 
    elif (p_val < significance_level) & (adf_stat < crit_val_5):
        linecolor = 'orange'
    elif (p_val < significance_level) & (adf_stat < crit_val_10):
        linecolor = 'red'
    else:
        linecolor = 'purple'
    sns.lineplot(x=final_df['year'], y=series, ax=ax, color=linecolor)
    ax.set_title(f'ADF Statistic {adf_stat:0.3f}, p-value: {p_val:0.3f}\nCritical Values 1%: {crit_val_1:0.3f}, 5%: {crit_val_5:0.3f}, 10%: {crit_val_10:0.3f}', fontsize=14)
    ax.set_ylabel(ylabel=title, fontsize=14)

visualize_adfuller_results(final_df['ER'].values, 'Exchange Rate', ax[0, 0])
visualize_adfuller_results(final_df['REER'].values, 'Real Effective Exchange Rate', ax[1, 0])
visualize_adfuller_results(final_df['Pop'].values, 'Malaysia Population', ax[0, 1])
visualize_adfuller_results(final_df['GDP'].values, 'Gross Domestic Produc', ax[1, 1])
visualize_adfuller_results(final_df['Export'].values, 'Total Export of Malaysia', ax[2, 0])

f.delaxes(ax[2, 1])
plt.tight_layout()
plt.show()

In [ ]:
# First Order Differencing
ts_diff = np.diff(final_df['Export'])
final_df['Export1'] = np.append([0], ts_diff)

ts_diff = np.diff(final_df['ER'])
final_df['ER'] = np.append([0], ts_diff)

ts_diff = np.diff(final_df['REER'])
final_df['REER'] = np.append([0], ts_diff)

ts_diff = np.diff(final_df['GDP'])
final_df['GDP'] = np.append([0], ts_diff)

# Reconduct the ADF test to check for the stationarity of data
f, ax = plt.subplots(nrows=2, ncols=2, figsize=(17, 13.5))
visualize_adfuller_results(final_df['ER'].values, 'Differenced (1. Order) \n Exchange Rate', ax[0, 0])
visualize_adfuller_results(final_df['REER'].values, 'Differenced (1. Order) \n Real Effective Exchange Rate', ax[1, 0])
visualize_adfuller_results(final_df['GDP'].values, ' Differenced (1. Order) \nGross Domestic Produc', ax[1, 1])
visualize_adfuller_results(final_df['Export1'].values, ' Differenced (1. Order) \nTotal Export of Malaysia', ax[0, 1])

# Data Correlation

In [ ]:
df_corr = final_df.copy()

# Create a copy of the DataFrame with the target variable dropped
df_corr = df_corr.drop(['Export', 'year'] , axis=1)

# Create a correlation matrix
corr_matrix = df_corr.corr()

# Create a mask to hide the upper triangular portion of the heatmap
mask = np.zeros_like(corr_matrix, dtype=np.bool_)
mask[np.triu_indices_from(mask)] = True

# Create a figure and axis object
fig, ax = plt.subplots(figsize=(10,10))

# Create a heatmap of the correlation matrix using seaborn
sns.heatmap(corr_matrix, cmap='viridis', annot=True, vmin=-1, vmax=1, mask=mask, fmt=".2f", square=True, ax=ax)

# Set the title of the plot
ax.set_title('Correlation Matrix')

# Show the plot
plt.show()

# Modeling
## Data Splitting into Training and Testing Set

In [ ]:
train_size = int(0.8 * len(final_df))
test_size = len(final_df) - train_size 

print(train_size, test_size)

univariate_df = final_df[['year', 'Export1']].copy()

train_data = univariate_df.iloc[:train_size, :]

x_train, y_train = pd.DataFrame(univariate_df.iloc[:train_size, 0]), pd.DataFrame(univariate_df.iloc[:train_size, 1])
x_test, y_test = pd.DataFrame(univariate_df.iloc[train_size:, 0]), pd.DataFrame(univariate_df.iloc[train_size:, 1])

In [ ]:
#plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Year')
plt.ylabel('Total Export Value')
plt.plot(x_train, y_train, 'green', label='Train data')
plt.plot(x_test, y_test, 'blue', label='Test data')
plt.legend()

## Autoregressive Integrated Moving Average (ARIMA) Model

### Identification of the best performance ARIMA model via auto-arima
Auto ARIM helps to automatically identify the optimal order for an ARIMA model.
The auto_arima function seeks to identify the most optimal parameters for an ARIMA model, and returns a fitted ARIMA model. This function is based on the commonly-used R function, forecast::auto.arima.

In [ ]:
model_autoARIMA = auto_arima(y_train, start_p=0, start_q=0, 
                             test ='adf',       # use adftest to find optimal 'd'
                              max_p=3, max_q=3, # maximum p and q
                              m=1,              # frequency of series
                              d=None,           # let model determine 'd'
                              seasonal=False,   # No Seasonality
                              start_P=0, 
                              D=0, 
                              trace=True,
                              error_action='ignore',  
                              suppress_warnings=True, 
                              stepwise=True)

print(model_autoARIMA.summary())
model_autoARIMA.plot_diagnostics(figsize=(15,8))
plt.show()

### Fit the data into the ARIMA (3, 2, 0) model

In [ ]:
# Fit model
model = ARIMA(y_train, order=(3,2,0))

model_fit = model.fit()
print(model_fit.summary())

In [ ]:
# Forecast the results using the test data
fc = model_fit.forecast(5, alpha=0.05)  # 95% conf
print(fc)

In [ ]:
# Make as pandas series
fc_series = pd.Series(fc)

# PLot graph to view the predicted and actual total export value
plt.figure(figsize=(10,5), dpi=100)
plt.plot(x_train, y_train, label='training data')
plt.plot(x_test, y_test, color = 'blue', label='Actual Stock Price')
plt.plot(x_test, fc_series, color = 'orange',label='Predicted Stock Price')
plt.title('Total Export Value of Malaysia')
plt.xlabel('Year')
plt.ylabel('Total Export Value (in billions USD)')
plt.legend(loc='upper left', fontsize=8)
plt.show()

### Peformance Evaluation

In [ ]:
rmse = math.sqrt(mean_squared_error(y_test, fc))
print('RMSE: '+str(rmse))
mape = np.mean(np.abs(y_test.iloc[:,0] - fc)/np.abs(y_test.iloc[:,0]))
print('MAPE: '+str(mape))

## Lazy Classifier
A quick estimation on the performance of other regressor models.

In [ ]:
# copy df
df_lc= final_df.copy()

# separate features and target variable
data = df_lc.drop(['Export1', 'Export'] , axis=1)
target = df_lc['Export1']


X, y = shuffle(data, target, random_state=13)
offset = int(X.shape[0] * 0.5)

X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]


# initialize LazyClassifier
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)

# fit and predict using multiple machine learning models
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

print(models)